# `nanoGPT`: Shakespeare

## Install / Setup

### Google Colab

```python
from google.colab import drive
drive.mount('/content/drive')
```

### First Time Running

We need to install `wordplay` and setup the Shakespeare dataset

This will need to be ran the first time you are running this notebook.

Following the

```python
!python3 -m pip install wordplay
```

you will need to restart your runtime (Runtime -> Restart runtime)

After this, you should be able to

```python
>>> import ngpt
>>> ngpt.__file__
'/content/nanoGPT/src/ngpt/__init__.py'
```

In [1]:
%%bash

python3 -c 'import wordplay; print(wordplay.__file__)' 2> '/dev/null'

if [[ $? -eq 0 ]]; then
    echo "Has wordplay installed. Nothing to do."
else
    echo "Does not have wordplay installed. Installing..."
    git clone 'https://github.com/saforem2/wordplay'
    python3 wordplay/data/shakespeare_char/prepare.py
    python3 -m pip install -e nanoGPT -vvv
fi

/Users/samforeman/projects/saforem2/wordplay/src/wordplay/__init__.py


Has wordplay installed. Nothing to do.


## Post Install

If installed correctly, you should be able to:

```python
>>> import wordplay
>>> wordplay.__file__
'/path/to/wordplay/src/wordplay/__init__.py'
```

In [2]:
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
os.environ['COLORTERM'] = 'truecolor'

In [3]:
%load_ext autoreload
%autoreload 2

import wordplay
from enrich import get_logger
log = get_logger(level='INFO')
#from rich import print
log.info(wordplay.__file__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[2024-02-06 00:12:54][INFO][1627752624:8] - /Users/samforeman/projects/saforem2/wordplay/src/wordplay/__init__.py


## Build Trainer

Explicitly, we:

1. `setup_torch(...)`
2. Build `cfg: DictConfig = get_config(...)`
3. Instnatiate `config: ExperimentConfig = instantiate(cfg)`
4. Build `trainer = Trainer(config)`

In [4]:
import os
import numpy as np
from ezpz import setup
from hydra.utils import instantiate
from wordplay.configs import get_config, PROJECT_ROOT
from wordplay.trainer import Trainer

HF_DATASETS_CACHE = PROJECT_ROOT.joinpath('.cache', 'huggingface')
HF_DATASETS_CACHE.mkdir(exist_ok=True, parents=True)

os.environ['HF_DATASETS_CACHE'] = HF_DATASETS_CACHE.as_posix()

rank = setup(
    framework='pytorch',
    backend='deepspeed',
    seed=1234,
)

cfg = get_config(
    [
        'data=shakespeare',
        'model=shakespeare',
        'optimizer=shakespeare',
        'train=shakespeare',
        'train.backend=deepspeed',
        'train.compile=false',
        'train.dtype=float32',
        'train.max_iters=2000',
        'train.log_interval=100',
        'train.eval_interval=500',
    ]
)
config = instantiate(cfg)

[2024-02-06 00:13:21][INFO][configs:72] - Setting HF_DATASETS_CACHE to /Users/samforeman/projects/saforem2/wordplay/.cache/huggingface/datasets

[2024-02-06 00:13:22][DEBUG][__init__:108] - Fira Sans already downloaded, continuing!

[2024-02-06 00:13:22][DEBUG][__init__:108] - Fira Sans Condensed already downloaded, continuing!

[2024-02-06 00:13:22][DEBUG][__init__:108] - IBM Plex Sans already downloaded, continuing!

[2024-02-06 00:13:22][DEBUG][__init__:108] - IBM Plex Sans Condensed already downloaded, continuing!

...

[2024-02-06 00:13:25][INFO][configs:308] - Loading val from /Users/samforeman/projects/saforem2/wordplay/data/shakespeare_char/val.bin

[2024-02-06 00:13:25][INFO][configs:283] - Rescaling GAS -> GAS // WORLD_SIZE = 1 // 1

[2024-02-06 00:13:25][INFO][configs:433] - Tokens per iteration: 16,384

[2024-02-06 00:13:25][INFO][configs:456] - Using self.ptdtype=torch.float32 on self.device_type='mps'

[2024-02-06 00:13:25][INFO][configs:462] - Initializing a new model from scratch

In [5]:
trainer = Trainer(config)

[2024-02-06 00:13:33][INFO][trainer:277] - Initializing a new model from scratch

[2024-02-06 00:13:33][INFO][model:255] - number of parameters: 10.65M

[2024-02-06 00:13:33][INFO][model:445] - num decayed parameter tensors: 26, with 10,740,096 parameters

[2024-02-06 00:13:33][INFO][model:449] - num non-decayed parameter tensors: 13, with 4,992 parameters

[2024-02-06 00:13:33][INFO][model:465] - using fused AdamW: False

...

[2024-02-06 00:13:35,827] [INFO] [config.py:988:print]   zero_enabled ................. False


[2024-02-06 00:13:35,827] [INFO] [config.py:988:print]   zero_force_ds_cpu_optimizer .. True


[2024-02-06 00:13:35,827] [INFO] [config.py:988:print]   zero_optimization_stage ...... 0


[2024-02-06 00:13:35,828] [INFO] [config.py:974:print_user_config]   json = {
    "dump_state": false, 
    "gradient_accumulation_steps": 1, 
    "wall_clock_breakdown": false, 
    "flops_profiler": {
        "enabled": false, 
        "profile_step": 1
    }, 
    "wandb": {
        "enabled": true, 
        "project": "WordPlay"
    }, 
    "train_micro_batch_size_per_gpu": 64, 
    "steps_per_print": 100
}


[rank0]:[W MPSFallback.mm:13] Warning: The operator 'c10d::broadcast_' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (function operator())


## Prompt (**prior** to training)

In [6]:
query = "What is an LLM?"
outputs = trainer.evaluate(
    query,
    num_samples=1,
    max_new_tokens=32,
    top_k=1,
    display=False
)
log.info(f"['prompt']: '{query}'")
log.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

[2024-02-06 00:15:20][INFO][4225705876:9] - ['prompt']: 'What is an LLM?'

[2024-02-06 00:15:20][INFO][4225705876:10] - ['response']:

What is an LLM??GGGGGVVVVVVVVVVVYYYYYYYYYCCCCCC

## Train Model

|  name  |       description            |
|:------:|:----------------------------:|
| `step` | Current training step        |
| `loss` | Loss value                   |
| `dt`   | Time per step (in **ms**)    |
| `sps`  | Samples per second           |
| `mtps` | (million) Tokens per sec     |
| `mfu`  | Model Flops utilization[^1]  |
^legend: #tbl-legend

[^1]: in units of A100 `bfloat16` peak FLOPS

In [7]:
trainer.config.device_type

'mps'

In [8]:
trainer.train()

  0%|          | 0/2000 [00:00<?, ?it/s]

                              Training Legend                               
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃    abbr    ┃ desc                                                        ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    step    │ Current training iteration                                  │
│    loss    │ Loss value                                                  │
│     dt     │ Elapsed time per training step (measured in **ms**)         │
│    dtf     │ Elapsed time per forward step (measured in **ms**)          │
│    dtb     │ Elapsed time per backward step (measured in **ms**)         │
│    sps     │ Samples per second                                          │
│    mtps    │ Tokens per second, measured in MEGA (1 x 10^6) tokens / sec │
│    mfu     │ Model flops utilization                                     │
│ train_loss │ Training loss value                                         │
│  val_loss  │ Validation loss value                                       │
└────────────┴─────────────────────────────────────────────────────────────┘

  0%|          | 1/2000 [00:44<24:45:30, 44.59s/it]

  0%|          | 2/2000 [00:45<10:27:01, 18.83s/it]

  0%|          | 3/2000 [00:45<5:45:22, 10.38s/it] 

...

[2024-02-06 00:28:34,236] [INFO] [logging.py:96:log_dist] [Rank 0] step=2000, skipped=0, lr=[0.000705685003441866, 0.000705685003441866], mom=[(0.9, 0.99), (0.9, 0.99)]


[2024-02-06 00:28:34,237] [INFO] [timer.py:260:stop] epoch=0/micro_step=2000/global_step=2000, RunningAvgSamplesPerSec=205.29878533900109, CurrSamplesPerSec=212.37335113953634, MemAllocated=0.9GB, MaxMemAllocated=6.33GB


[2024-02-06 00:28:34][INFO][trainer:768] - step=2000 loss=1.1520 dt=303.4430 dtf=55.4990 dtb=181.7013 sps=3.2955 mtps=0.0540 mfu=1.1852 train_loss=1.1642 val_loss=1.4781

100%|██████████| 2000/2000 [13:13<00:00,  3.29it/s]

100%|██████████| 2000/2000 [13:13<00:00,  2.52it/s]

## Evaluate Model

In [9]:
import time

query = "What is an LLM?"
t0 = time.perf_counter()
outputs = trainer.evaluate(
    query,
    num_samples=1,
    max_new_tokens=128,
    top_k=2,
    display=False
)
log.info(f'took: {time.perf_counter() - t0:.4f}s')
log.info(f"['prompt']: '{query}'")
log.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

[2024-02-06 00:32:51][INFO][3629240533:12] - took: 224.8553s

[2024-02-06 00:32:51][INFO][3629240533:13] - ['prompt']: 'What is an LLM?'

[2024-02-06 00:32:51][INFO][3629240533:14] - ['response']:

What is an LLM? the man that hath stood the strength
Of the seated with the stars and streams
As the world the wars and storms and the walls
Of